In [4]:
import requests
import json

In [5]:
def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    #response = requests.post(automat_url,json=message['message'])
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    #print(response.json())
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def ontology(message):
    url = 'https://stars-app.renci.org/sparql-kp/query'
    return post('ontology',url,message)
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

def aragorn(message, coalesce_type):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

In [11]:
subq={
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "id": "HP:0040301"
                },
                "b": {
                    "category": "biolink:NamedThing"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }}}

In [12]:
r = ontology(subq)

In [13]:
r

{'message': {'query_graph': {'nodes': {'a': {'id': 'HP:0040301'},
    'b': {'category': 'biolink:NamedThing'}},
   'edges': {'ab': {'subject': 'a', 'object': 'b'}}},
  'knowledge_graph': {'nodes': {'HP:0000079': {'name': 'Abnormality of the urinary system',
     'category': ['biolink:NamedThing']},
    'HP:0001939': {'name': 'Abnormality of metabolism/homeostasis',
     'category': ['biolink:NamedThing']},
    'OBO:CARO_0000011': {'name': 'connected anatomical system',
     'category': ['biolink:NamedThing']},
    'CHEBI:16646': {'name': 'carbohydrate',
     'category': ['biolink:NamedThing']},
    'BFO:0000003': {'name': 'occurrent', 'category': ['biolink:NamedThing']},
    'UBERON:0001062': {'name': 'anatomical entity',
     'category': ['biolink:NamedThing']},
    'OBO:CARO_0000003': {'name': 'anatomical structure',
     'category': ['biolink:NamedThing']},
    'CHEBI:33608': {'name': 'hydrogen molecular entity',
     'category': ['biolink:NamedThing']},
    'UBERON:0000061': {'name

In [9]:
rs = strider(subq)

In [10]:
rs

{'message': {'query_graph': {'nodes': {'b': {'id': None,
     'category': ['biolink:ChemicalSubstance'],
     'is_set': False},
    'a': {'id': ['CHEBI:3002'],
     'category': ['biolink:ChemicalSubstance'],
     'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': ['biolink:subclass_of'],
     'relation': None}}},
  'knowledge_graph': {'nodes': {}, 'edges': {}},
  'results': []},
 'logs': [{'timestamp': '2021-04-19T15:34:26.862940',
   'level': 'DEBUG',
   'code': None,
   'message': '{"message": "Received response from KP", "url": "https://api.bte.ncats.io/v1/query", "request": {"message": {"query_graph": {"nodes": {"a": {"category": "biolink:ChemicalSubstance", "is_set": false, "id": "PUBCHEM.COMPOUND:21700"}, "b": {"category": "biolink:Metabolite", "is_set": false}}, "edges": {"ab": {"subject": "a", "object": "b", "predicate": "biolink:subclass_of"}}}}}, "response": {"message": {"query_graph": {"nodes": {"a": {"category": "biolink:ChemicalSub